In [1]:
import os
import math
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.optim as optim
import numpy as np
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_fscore_support
import matplotlib.pyplot as plt

class ResidualDataset(Dataset):
    def __init__(self, csv_file, clean_base, adv_base, split='train'):
        self.df = pd.read_csv(csv_file, usecols=['rel_path', 'detect_split'])
        self.df = self.df[self.df['detect_split'] == split].reset_index(drop=True)
        self.clean_base = clean_base
        self.adv_base = adv_base

        self.files = []
        self.labels = []
        for _, row in self.df.iterrows():
            rel_path_pt = row['rel_path'].replace('.jpg', '.pt')
            # clean
            self.files.append(os.path.join(self.clean_base, rel_path_pt.lstrip('/')))
            self.labels.append(0)
            # adversarial
            self.files.append(os.path.join(self.adv_base, rel_path_pt.lstrip('/')))
            self.labels.append(1)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = self.files[idx]
        label = self.labels[idx]
        tensor = torch.load(path)
        tensor = tensor.float() / 255.0
        return tensor, label

def evaluate_on_loader(model, loader, device='cuda'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total if total > 0 else 0.0
    return acc, total


def evaluate_on_loader_with_loss(model, loader, criterion, device='cuda'):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    avg_loss = running_loss / total if total > 0 else 0.0
    acc = correct / total if total > 0 else 0.0
    return acc, total, avg_loss


def evaluate_test(model=None, csv_file=None,
                  clean_base='../03-ExtractCharacteristics/jpg_residual-clean/256x256/',
                  adv_base='../03-ExtractCharacteristics/jpg_residual/256x256/cw/',
                  batch_size=16,
                  device='cuda',
                  model_path=None,
                  roc_path='./results/roc_test.png'):
    assert csv_file is not None, "csv_file must be provided"

    if model is None and model_path is not None:
        model = load_model_state(model_path, device=device)
    assert model is not None, "Either a model or model_path must be provided"

    test_ds = ResidualDataset(csv_file, clean_base, adv_base, split='test')
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []  
    
    softmax = nn.Softmax(dim=1)
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            probs = softmax(outputs)[:, 1]  
            preds = torch.argmax(outputs, dim=1)

            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_probs = np.array(all_probs)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    acc = (all_preds == all_labels).mean() if all_labels.size > 0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=[0, 1]) if all_labels.size > 0 else np.zeros((2,2),dtype=int)

    auc = None
    roc_data = None
    try:
        if len(np.unique(all_labels)) == 2:
            auc = roc_auc_score(all_labels, all_probs)
            fpr, tpr, thresholds = roc_curve(all_labels, all_probs)
            roc_data = (fpr, tpr, thresholds)
            # plot ROC
            os.makedirs(os.path.dirname(roc_path), exist_ok=True)
            plt.figure()
            plt.plot(fpr, tpr)
            plt.plot([0,1], [0,1], linestyle='--')
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC curve (AUC = {auc:.4f})')
            plt.grid(True)
            plt.savefig(roc_path)
            plt.close()
        else:
            auc = None
            roc_data = None
    except Exception as e:
        auc = None
        roc_data = None

    pr, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)

    results = {
        'accuracy': float(acc),
        'total': int(all_labels.size),
        'confusion_matrix': cm,
        'auc': float(auc) if auc is not None else None,
        'roc_curve': roc_data,
        'precision_recall_f1': (float(pr), float(rec), float(f1)),
        'roc_path': roc_path if roc_data is not None else None,
    }

    # Print a compact summary
    print(f"Test Accuracy: {results['accuracy']:.4f} over {results['total']} samples")
    print('Confusion Matrix (rows=true, cols=pred):')
    print(results['confusion_matrix'])
    if results['auc'] is not None:
        print(f"AUC-ROC: {results['auc']:.4f} (ROC plot saved to: {results['roc_path']})")
    else:
        print('AUC-ROC: not available (need both classes present in test set)')
    prc = results['precision_recall_f1']
    print(f"Precision: {prc[0]:.4f}, Recall: {prc[1]:.4f}, F1: {prc[2]:.4f}")

    return results

def save_model_state(model, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(model.state_dict(), path)


def load_model_state(path, device='cuda'):
    model = models.resnet18(weights=None)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(model.fc.in_features, 2)

    map_location = torch.device(device)
    checkpoint = torch.load(path, map_location=map_location)

    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        state = checkpoint['model_state_dict']
    else:
        state = checkpoint

    model.load_state_dict(state)
    model = model.to(device)
    model.eval()
    return model

def train_detector(csv_file,
                   clean_base='../03-ExtractCharacteristics/jpg_residual-clean/256x256/',
                   adv_base='../03-ExtractCharacteristics/jpg_residual/256x256/cw/',
                   batch_size=4,
                   num_epochs=50,
                   lr=1e-3,
                   device='cuda',
                   model_save_path='./saved_models/best_detector.pth',
                   save_best_only=True):
    train_ds = ResidualDataset(csv_file, clean_base, adv_base, split='train')
    val_ds = ResidualDataset(csv_file, clean_base, adv_base, split='val')

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, pin_memory=True)

    model = models.resnet18(weights=None)

    model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)

    best_val_acc = 0.0
    best_val_loss = math.inf
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total if total > 0 else 0.0
        train_acc = correct / total if total > 0 else 0.0

        val_acc, val_total, val_loss = evaluate_on_loader_with_loss(model, val_loader, criterion, device=device)

        scheduler.step(1 - val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}, Val Loss={val_loss:.4f}")

        save_current = False
        if val_acc > best_val_acc:
            save_current = True
        elif val_acc == best_val_acc and val_loss < best_val_loss:
            save_current = True

        if save_current:
            best_val_acc = val_acc
            best_val_loss = val_loss
            
            if save_best_only:
                save_model_state(model, model_save_path)
                print(f"Saved best model (val_acc={best_val_acc:.4f}, val_loss={best_val_loss:.4f}) to: {model_save_path}")

    if not save_best_only:
        save_model_state(model, model_save_path)
        print(f"Saved final model to: {model_save_path}")

    return None


In [2]:
img_size = 32
attack_methods = ['fgsm','bim','pgd','df','cw']
for attack_method in attack_methods:
    print(f'Model Training - {img_size}x{img_size} - {attack_method} \n ')
    train_detector(csv_file='/kaggle/input/splits/common_success_detect_split.csv',
                   clean_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/clean',
                   adv_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/{attack_method}',
                   batch_size=32,
                   num_epochs=50,
                   lr=1e-3,
                   device='cuda',
                   model_save_path=f'./saved_models/best_detector_{img_size}x{img_size}_{attack_method}.pth',
                   save_best_only=True)
    print('Done. \n')

Model Training - 32x32 - fgsm 
 
Epoch 1/50: Train Loss=0.1210, Train Acc=0.9599, Val Acc=0.5000, Val Loss=8.1504
Saved best model (val_acc=0.5000, val_loss=8.1504) to: ./saved_models/best_detector_32x32_fgsm.pth
Epoch 2/50: Train Loss=0.0353, Train Acc=0.9918, Val Acc=0.5000, Val Loss=5.9487
Saved best model (val_acc=0.5000, val_loss=5.9487) to: ./saved_models/best_detector_32x32_fgsm.pth
Epoch 3/50: Train Loss=0.0023, Train Acc=1.0000, Val Acc=0.5000, Val Loss=6.5063
Epoch 4/50: Train Loss=0.0006, Train Acc=1.0000, Val Acc=1.0000, Val Loss=0.0070
Saved best model (val_acc=1.0000, val_loss=0.0070) to: ./saved_models/best_detector_32x32_fgsm.pth
Epoch 5/50: Train Loss=0.0003, Train Acc=1.0000, Val Acc=1.0000, Val Loss=0.0002
Saved best model (val_acc=1.0000, val_loss=0.0002) to: ./saved_models/best_detector_32x32_fgsm.pth
Epoch 6/50: Train Loss=0.0002, Train Acc=1.0000, Val Acc=1.0000, Val Loss=0.0001
Saved best model (val_acc=1.0000, val_loss=0.0001) to: ./saved_models/best_detector_3

In [3]:
img_size = 64
attack_methods = ['fgsm','bim','pgd','df','cw']
for attack_method in attack_methods:
    print(f'Model Training - {img_size}x{img_size} - {attack_method} \n ')
    train_detector(csv_file='/kaggle/input/splits/common_success_detect_split.csv',
                   clean_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/clean',
                   adv_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/{attack_method}',
                   batch_size=32,
                   num_epochs=50,
                   lr=1e-3,
                   device='cuda',
                   model_save_path=f'./saved_models/best_detector_{img_size}x{img_size}_{attack_method}.pth',
                   save_best_only=True)
    print('Done. \n')

Model Training - 64x64 - fgsm 
 
Epoch 1/50: Train Loss=0.0270, Train Acc=0.9856, Val Acc=0.5000, Val Loss=5.4084
Saved best model (val_acc=0.5000, val_loss=5.4084) to: ./saved_models/best_detector_64x64_fgsm.pth
Epoch 2/50: Train Loss=0.0013, Train Acc=1.0000, Val Acc=0.5000, Val Loss=6.5196
Epoch 3/50: Train Loss=0.0427, Train Acc=0.9949, Val Acc=0.5000, Val Loss=4.8773
Saved best model (val_acc=0.5000, val_loss=4.8773) to: ./saved_models/best_detector_64x64_fgsm.pth
Epoch 4/50: Train Loss=0.0011, Train Acc=1.0000, Val Acc=0.9753, Val Loss=0.0695
Saved best model (val_acc=0.9753, val_loss=0.0695) to: ./saved_models/best_detector_64x64_fgsm.pth
Epoch 5/50: Train Loss=0.0139, Train Acc=0.9979, Val Acc=1.0000, Val Loss=0.0001
Saved best model (val_acc=1.0000, val_loss=0.0001) to: ./saved_models/best_detector_64x64_fgsm.pth
Epoch 6/50: Train Loss=0.0024, Train Acc=0.9990, Val Acc=1.0000, Val Loss=0.0001
Saved best model (val_acc=1.0000, val_loss=0.0001) to: ./saved_models/best_detector_6

In [4]:
img_size = 128
attack_methods = ['fgsm','bim','pgd','df','cw']
for attack_method in attack_methods:
    print(f'Model Training - {img_size}x{img_size} - {attack_method} \n ')
    train_detector(csv_file='/kaggle/input/splits/common_success_detect_split.csv',
                   clean_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/clean',
                   adv_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/{attack_method}',
                   batch_size=32,
                   num_epochs=50,
                   lr=1e-3,
                   device='cuda',
                   model_save_path=f'./saved_models/best_detector_{img_size}x{img_size}_{attack_method}.pth',
                   save_best_only=True)
    print('Done. \n')

Model Training - 128x128 - fgsm 
 
Epoch 1/50: Train Loss=0.0748, Train Acc=0.9630, Val Acc=0.5000, Val Loss=5.3436
Saved best model (val_acc=0.5000, val_loss=5.3436) to: ./saved_models/best_detector_128x128_fgsm.pth
Epoch 2/50: Train Loss=0.0266, Train Acc=0.9918, Val Acc=0.5000, Val Loss=3.4101
Saved best model (val_acc=0.5000, val_loss=3.4101) to: ./saved_models/best_detector_128x128_fgsm.pth
Epoch 3/50: Train Loss=0.0022, Train Acc=1.0000, Val Acc=0.5000, Val Loss=4.5751
Epoch 4/50: Train Loss=0.0037, Train Acc=0.9990, Val Acc=1.0000, Val Loss=0.0001
Saved best model (val_acc=1.0000, val_loss=0.0001) to: ./saved_models/best_detector_128x128_fgsm.pth
Epoch 5/50: Train Loss=0.0058, Train Acc=0.9969, Val Acc=0.9259, Val Loss=0.5230
Epoch 6/50: Train Loss=0.0008, Train Acc=1.0000, Val Acc=1.0000, Val Loss=0.0254
Epoch 7/50: Train Loss=0.0007, Train Acc=1.0000, Val Acc=1.0000, Val Loss=0.0000
Saved best model (val_acc=1.0000, val_loss=0.0000) to: ./saved_models/best_detector_128x128_fgs

In [3]:
img_size = 256
attack_methods = ['fgsm','bim','pgd','df','cw']
for attack_method in attack_methods:
    print(f'Model Training - {img_size}x{img_size} - {attack_method} \n ')
    train_detector(csv_file='/kaggle/input/splits/common_success_detect_split.csv',
                   clean_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/clean',
                   adv_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/{attack_method}',
                   batch_size=32,
                   num_epochs=50,
                   lr=1e-3,
                   device='cuda',
                   model_save_path=f'./saved_models/best_detector_{img_size}x{img_size}_{attack_method}.pth',
                   save_best_only=True)
    print('Done. \n')

Model Training - 256x256 - fgsm 
 
Epoch 1/50: Train Loss=0.1030, Train Acc=0.9609, Val Acc=0.5000, Val Loss=5.5890
Saved best model (val_acc=0.5000, val_loss=5.5890) to: ./saved_models/best_detector_256x256_fgsm.pth
Epoch 2/50: Train Loss=0.0304, Train Acc=0.9907, Val Acc=0.5000, Val Loss=3.2669
Saved best model (val_acc=0.5000, val_loss=3.2669) to: ./saved_models/best_detector_256x256_fgsm.pth
Epoch 3/50: Train Loss=0.0075, Train Acc=0.9979, Val Acc=0.5000, Val Loss=4.7336
Epoch 4/50: Train Loss=0.0046, Train Acc=0.9990, Val Acc=1.0000, Val Loss=0.0009
Saved best model (val_acc=1.0000, val_loss=0.0009) to: ./saved_models/best_detector_256x256_fgsm.pth
Epoch 5/50: Train Loss=0.0026, Train Acc=0.9990, Val Acc=1.0000, Val Loss=0.0001
Saved best model (val_acc=1.0000, val_loss=0.0001) to: ./saved_models/best_detector_256x256_fgsm.pth
Epoch 6/50: Train Loss=0.0002, Train Acc=1.0000, Val Acc=1.0000, Val Loss=0.0001
Saved best model (val_acc=1.0000, val_loss=0.0001) to: ./saved_models/best_

In [4]:
img_size = 512
attack_methods = ['fgsm','bim','pgd','df','cw']
for attack_method in attack_methods:
    print(f'Model Training - {img_size}x{img_size} - {attack_method} \n ')
    train_detector(csv_file='/kaggle/input/splits/common_success_detect_split.csv',
                   clean_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/clean',
                   adv_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/{attack_method}',
                   batch_size=32,
                   num_epochs=50,
                   lr=1e-3,
                   device='cuda',
                   model_save_path=f'./saved_models/best_detector_{img_size}x{img_size}_{attack_method}.pth',
                   save_best_only=True)
    print('Done. \n')

Model Training - 512x512 - fgsm 
 
Epoch 1/50: Train Loss=0.1674, Train Acc=0.9609, Val Acc=0.5000, Val Loss=2.8731
Saved best model (val_acc=0.5000, val_loss=2.8731) to: ./saved_models/best_detector_512x512_fgsm.pth
Epoch 2/50: Train Loss=0.0557, Train Acc=0.9856, Val Acc=0.5000, Val Loss=2.6258
Saved best model (val_acc=0.5000, val_loss=2.6258) to: ./saved_models/best_detector_512x512_fgsm.pth
Epoch 3/50: Train Loss=0.0232, Train Acc=0.9949, Val Acc=0.5000, Val Loss=3.1980
Epoch 4/50: Train Loss=0.0217, Train Acc=0.9938, Val Acc=0.6296, Val Loss=0.6038
Saved best model (val_acc=0.6296, val_loss=0.6038) to: ./saved_models/best_detector_512x512_fgsm.pth
Epoch 5/50: Train Loss=0.0225, Train Acc=0.9949, Val Acc=0.9198, Val Loss=0.4864
Saved best model (val_acc=0.9198, val_loss=0.4864) to: ./saved_models/best_detector_512x512_fgsm.pth
Epoch 6/50: Train Loss=0.0200, Train Acc=0.9949, Val Acc=0.9938, Val Loss=0.0373
Saved best model (val_acc=0.9938, val_loss=0.0373) to: ./saved_models/best_

In [2]:
img_size = 1024
attack_methods = ['fgsm','bim','pgd','df','cw']
for attack_method in attack_methods:
    print(f'Model Training - {img_size}x{img_size} - {attack_method} \n ')
    train_detector(csv_file='/kaggle/input/splits/common_success_detect_split.csv',
                   clean_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/clean',
                   adv_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/{attack_method}',
                   batch_size=32,
                   num_epochs=50,
                   lr=1e-3,
                   device='cuda',
                   model_save_path=f'./saved_models/best_detector_{img_size}x{img_size}_{attack_method}.pth',
                   save_best_only=True)
    print('Done. \n')

Model Training - 1024x1024 - fgsm 
 
Epoch 1/50: Train Loss=0.1422, Train Acc=0.9547, Val Acc=0.5000, Val Loss=3.1972
Saved best model (val_acc=0.5000, val_loss=3.1972) to: ./saved_models/best_detector_1024x1024_fgsm.pth
Epoch 2/50: Train Loss=0.0280, Train Acc=0.9938, Val Acc=0.5000, Val Loss=2.6128
Saved best model (val_acc=0.5000, val_loss=2.6128) to: ./saved_models/best_detector_1024x1024_fgsm.pth
Epoch 3/50: Train Loss=0.0267, Train Acc=0.9938, Val Acc=0.5000, Val Loss=2.7896
Epoch 4/50: Train Loss=0.0252, Train Acc=0.9969, Val Acc=1.0000, Val Loss=0.0056
Saved best model (val_acc=1.0000, val_loss=0.0056) to: ./saved_models/best_detector_1024x1024_fgsm.pth
Epoch 5/50: Train Loss=0.0216, Train Acc=0.9959, Val Acc=1.0000, Val Loss=0.0009
Saved best model (val_acc=1.0000, val_loss=0.0009) to: ./saved_models/best_detector_1024x1024_fgsm.pth
Epoch 6/50: Train Loss=0.0507, Train Acc=0.9877, Val Acc=1.0000, Val Loss=0.0025
Epoch 7/50: Train Loss=0.0219, Train Acc=0.9959, Val Acc=1.0000, 

In [11]:
import os
import zipfile
import hashlib

# Define paths
folder_path = '/kaggle/working/saved_models'
zip_path = '/kaggle/working/saved_models.zip'
hash_path = '/kaggle/working/saved_models.sha256'

if os.path.exists(folder_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Keep relative structure inside the zip
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)
    print(f"Zipped all contents to {zip_path}")
else:
    raise FileNotFoundError(f"Folder not found: {folder_path}")

def sha256sum(filename, block_size=65536):
    h = hashlib.sha256()
    with open(filename, 'rb') as f:
        for block in iter(lambda: f.read(block_size), b''):
            h.update(block)
    return h.hexdigest()

hash_value = sha256sum(zip_path)

with open(hash_path, 'w') as f:
    f.write(f"{hash_value}  {os.path.basename(zip_path)}\n")

print(f"SHA256 hash saved to {hash_path}")
print(f"Hash: {hash_value}")


Zipped all contents to /kaggle/working/saved_models.zip
SHA256 hash saved to /kaggle/working/saved_models.sha256
Hash: b1711da841aa5a328cd5cb23819b407604bc3f0e66589ee191da4909d9b86f18


In [2]:
# --- Add these imports at the top of your script ---
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix, 
    roc_auc_score, 
    roc_curve, 
    precision_recall_fscore_support
)
from pathlib import Path
# --- Make sure your other imports (like ResidualDataset, load_model_state) are also present ---
# from your_dataset_module import ResidualDataset
# from your_model_utils import load_model_state
# --------------------------------------------------


def evaluate_test(model=None, csv_file=None,
                  clean_base='../03-ExtractCharacteristics/jpg_residual-clean/256x256/',
                  adv_base='../03-ExtractCharacteristics/jpg_residual/256x256/cw/',
                  batch_size=16,
                  device='cuda',
                  model_path=None,
                  roc_path='./results/roc_test.png',
                  output_csv_path=None,  
                  return_predictions=False 
                 ):
    assert csv_file is not None, "csv_file must be provided"

    if model is None and model_path is not None:
        model = load_model_state(model_path, device=device)
    assert model is not None, "Either a model or model_path must be provided"

    test_ds = ResidualDataset(csv_file, clean_base, adv_base, split='test')
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []  
    
    softmax = nn.Softmax(dim=1)
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            probs = softmax(outputs)[:, 1] 
            preds = torch.argmax(outputs, dim=1)

            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_probs = np.array(all_probs)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    acc = (all_preds == all_labels).mean() if all_labels.size > 0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=[0, 1]) if all_labels.size > 0 else np.zeros((2,2),dtype=int)

    auc = None
    roc_data = None
    try:
        if len(np.unique(all_labels)) == 2:
            auc = roc_auc_score(all_labels, all_probs)
            fpr, tpr, thresholds = roc_curve(all_labels, all_probs)
            roc_data = (fpr, tpr, thresholds)
            # plot ROC
            os.makedirs(os.path.dirname(roc_path), exist_ok=True)
            plt.figure()
            plt.plot(fpr, tpr)
            plt.plot([0,1], [0,1], linestyle='--')
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC curve (AUC = {auc:.4f})')
            plt.grid(True)
            plt.savefig(roc_path)
            plt.close()
        else:
            auc = None
            roc_data = None
    except Exception as e:
        auc = None
        roc_data = None

    pr, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)

    results = {
        'accuracy': float(acc),
        'total': int(all_labels.size),
        'confusion_matrix': cm,
        'auc': float(auc) if auc is not None else None,
        'roc_curve': roc_data,
        'precision_recall_f1': (float(pr), float(rec), float(f1)),
        'roc_path': roc_path if roc_data is not None else None,
    }

    # Print a compact summary
    print(f"Test Accuracy: {results['accuracy']:.4f} over {results['total']} samples")
    print('Confusion Matrix (rows=true, cols=pred):')
    print(results['confusion_matrix'])
    if results['auc'] is not None:
        print(f"AUC-ROC: {results['auc']:.4f} (ROC plot saved to: {results['roc_path']})")
    else:
        print('AUC-ROC: not available (need both classes present in test set)')
    prc = results['precision_recall_f1']
    print(f"Precision: {prc[0]:.4f}, Recall: {prc[1]:.4f}, F1: {prc[2]:.4f}")

    # --- Save results to CSV (Added block) ---
    if output_csv_path is not None:
        csv_path = Path(output_csv_path)
        csv_path.parent.mkdir(parents=True, exist_ok=True)

        # Prepare metrics for CSV
        pr, rec, f1 = results['precision_recall_f1']
        cm_to_save = results['confusion_matrix']
        
        # Handle case where cm might be empty if all_labels.size == 0
        if cm_to_save.size == 4:
            # Ensure native int types for csv
            tn, fp, fn, tp = map(int, cm_to_save.ravel()) 
        else:
            tn, fp, fn, tp = 0, 0, 0, 0

        metrics_to_save = {
            'accuracy': results['accuracy'],
            'precision': pr,
            'recall': rec,
            'f1': f1,
            'auc': results['auc'],
            'total_samples': results['total'],
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'tp': tp,
            'roc_path': results['roc_path']
        }
        
        metrics_df = pd.DataFrame([metrics_to_save])
        metrics_df.to_csv(csv_path, index=False)
        print(f"Metrics saved to: {csv_path}")

        if return_predictions:
            preds_path = csv_path.with_name(csv_path.stem + "_predictions.csv")
            preds_df = pd.DataFrame({
                "y_true": all_labels,
                "y_pred": all_preds,
                "y_score_class1": all_probs # Using a more specific name
            })
            preds_df.to_csv(preds_path, index=False)
            print(f"Predictions saved to: {preds_path}")

    return None

In [3]:
attack_methods = ['fgsm', 'bim', 'pgd', 'df', 'cw']
datasets = ['fgsm', 'bim', 'pgd', 'df', 'cw']
img_sizes = [32, 64, 128, 256, 512, 1024]
classifier = 'adam'

for img_size in img_sizes:
    print (f'Image Size: {img_size}\n')
    for dataset in datasets:
        print (f'Dataset: {dataset}\n')
        for attack_method in attack_methods:
            print (f'Attack Method: {attack_method}\n')
            evaluate_test(model=None,
                          csv_file='/kaggle/input/splits/common_success_detect_split.csv',
                          clean_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/clean',
                          adv_base=f'/kaggle/input/jpg-residual-100/jpg_residual-100/{img_size}x{img_size}/{dataset}',
                          batch_size=32,
                          device='cuda',
                          model_path=f'./saved_models/best_detector_{img_size}x{img_size}_{attack_method}.pth',
                          roc_path=f'./eval-TEST/{attack_method}-{dataset}-{img_size}-{classifier}_ROC.png',
                          output_csv_path=f'./eval-TEST/{attack_method}-{dataset}-{img_size}-{classifier}_test_result.csv',
                          return_predictions=False)
            print('------------------------------\n')
        print ('++++++++++++++++++++++++++++++++\n')
    print ('=================================\n')



Image Size: 32

Dataset: fgsm

Attack Method: fgsm

Test Accuracy: 1.0000 over 164 samples
Confusion Matrix (rows=true, cols=pred):
[[82  0]
 [ 0 82]]
AUC-ROC: 1.0000 (ROC plot saved to: ./eval-TEST/fgsm-fgsm-32-adam_ROC.png)
Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Metrics saved to: eval-TEST/fgsm-fgsm-32-adam_test_result.csv
------------------------------

Attack Method: bim

Test Accuracy: 0.9939 over 164 samples
Confusion Matrix (rows=true, cols=pred):
[[82  0]
 [ 1 81]]
AUC-ROC: 1.0000 (ROC plot saved to: ./eval-TEST/bim-fgsm-32-adam_ROC.png)
Precision: 1.0000, Recall: 0.9878, F1: 0.9939
Metrics saved to: eval-TEST/bim-fgsm-32-adam_test_result.csv
------------------------------

Attack Method: pgd

Test Accuracy: 1.0000 over 164 samples
Confusion Matrix (rows=true, cols=pred):
[[82  0]
 [ 0 82]]
AUC-ROC: 1.0000 (ROC plot saved to: ./eval-TEST/pgd-fgsm-32-adam_ROC.png)
Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Metrics saved to: eval-TEST/pgd-fgsm-32-adam_test_result.csv
--

In [5]:
import hashlib
import os
from pathlib import Path
import zipfile
from datetime import datetime

# -------------------------------
# Config
# -------------------------------
input_dir = Path("/kaggle/working/eval-TEST")
output_zip = Path("/kaggle/working/evals-TEST.zip")
manifest_path = Path("/kaggle/working/manifest-evals-TEST-sha256.txt")
chunk_size = 8 * 1024 * 1024  # 8 MB
skip_files = {output_zip.name, manifest_path.name}

# -------------------------------
# Helper: compute SHA256
# -------------------------------
def sha256_of_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest()

# -------------------------------
# Collect files to zip
# -------------------------------
files_to_add = []
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        full = Path(root) / fname
        rel = full.relative_to(input_dir)
        rel_str = str(rel).replace(os.sep, "/")
        if rel.name in skip_files:
            continue
        files_to_add.append((full, rel))

if not files_to_add:
    print(f"No files found in {input_dir} to archive.")
else:
    # -------------------------------
    # Create zip
    # -------------------------------
    print(f"Creating zip: {output_zip}")
    with zipfile.ZipFile(output_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for full, rel in files_to_add:
            arcname = str(rel).replace(os.sep, "/")
            print(f"  adding: {arcname}")
            zf.write(full, arcname=arcname)

    # -------------------------------
    # Create manifest
    # -------------------------------
    print(f"Computing per-file SHA-256 and writing manifest: {manifest_path}")
    with manifest_path.open("w", encoding="utf-8") as mf:
        mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")
        mf.write(f"# input_dir: {input_dir}\n")
        mf.write("# format: <sha256>  <relative/path>\n")
        for full, rel in files_to_add:
            rel_unix = str(rel).replace(os.sep, "/")
            h = sha256_of_file(full)
            size = full.stat().st_size
            mf.write(f"{h}  {rel_unix}  # {size} bytes\n")

    # -------------------------------
    # Compute zip SHA256
    # -------------------------------
    zip_hash = sha256_of_file(output_zip)
    ziphash_path = output_zip.with_suffix(output_zip.suffix + ".sha256")
    with ziphash_path.open("w", encoding="utf-8") as zh:
        zh.write(f"{zip_hash}  {output_zip.name}\n")

    print("Done.")
    print(f"Created: {output_zip}")
    print(f"Created: {manifest_path}")
    print(f"Created: {ziphash_path}")


Creating zip: /kaggle/working/evals-TEST.zip
  adding: df-df-1024-adam_test_result.csv
  adding: fgsm-df-64-adam_ROC.png
  adding: df-pgd-256-adam_test_result.csv
  adding: pgd-bim-128-adam_ROC.png
  adding: df-fgsm-256-adam_test_result.csv
  adding: bim-df-32-adam_ROC.png
  adding: cw-pgd-32-adam_test_result.csv
  adding: fgsm-bim-64-adam_test_result.csv
  adding: cw-pgd-256-adam_ROC.png
  adding: fgsm-bim-512-adam_test_result.csv
  adding: pgd-cw-1024-adam_test_result.csv
  adding: pgd-bim-32-adam_test_result.csv
  adding: pgd-fgsm-512-adam_test_result.csv
  adding: pgd-bim-128-adam_test_result.csv
  adding: cw-cw-512-adam_ROC.png
  adding: df-bim-128-adam_ROC.png
  adding: fgsm-bim-128-adam_ROC.png
  adding: cw-bim-64-adam_ROC.png
  adding: fgsm-fgsm-32-adam_test_result.csv
  adding: bim-df-512-adam_test_result.csv
  adding: fgsm-cw-256-adam_ROC.png
  adding: cw-cw-32-adam_ROC.png
  adding: pgd-pgd-512-adam_test_result.csv
  adding: bim-df-512-adam_ROC.png
  adding: fgsm-fgsm-32-ada

In [6]:
import hashlib
import os
from pathlib import Path
import zipfile
from datetime import datetime

# -------------------------------
# Config
# -------------------------------
input_dir = Path("/kaggle/working/saved_models")
output_zip = Path("/kaggle/working/saved_models.zip")
manifest_path = Path("/kaggle/working/manifest-saved_models-sha256.txt")
chunk_size = 8 * 1024 * 1024  # 8 MB
skip_files = {output_zip.name, manifest_path.name}

# -------------------------------
# Helper: compute SHA256
# -------------------------------
def sha256_of_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest()

# -------------------------------
# Collect files to zip
# -------------------------------
files_to_add = []
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        full = Path(root) / fname
        rel = full.relative_to(input_dir)
        rel_str = str(rel).replace(os.sep, "/")
        if rel.name in skip_files:
            continue
        files_to_add.append((full, rel))

if not files_to_add:
    print(f"No files found in {input_dir} to archive.")
else:
    # -------------------------------
    # Create zip
    # -------------------------------
    print(f"Creating zip: {output_zip}")
    with zipfile.ZipFile(output_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for full, rel in files_to_add:
            arcname = str(rel).replace(os.sep, "/")
            print(f"  adding: {arcname}")
            zf.write(full, arcname=arcname)

    # -------------------------------
    # Create manifest
    # -------------------------------
    print(f"Computing per-file SHA-256 and writing manifest: {manifest_path}")
    with manifest_path.open("w", encoding="utf-8") as mf:
        mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")
        mf.write(f"# input_dir: {input_dir}\n")
        mf.write("# format: <sha256>  <relative/path>\n")
        for full, rel in files_to_add:
            rel_unix = str(rel).replace(os.sep, "/")
            h = sha256_of_file(full)
            size = full.stat().st_size
            mf.write(f"{h}  {rel_unix}  # {size} bytes\n")

    # -------------------------------
    # Compute zip SHA256
    # -------------------------------
    zip_hash = sha256_of_file(output_zip)
    ziphash_path = output_zip.with_suffix(output_zip.suffix + ".sha256")
    with ziphash_path.open("w", encoding="utf-8") as zh:
        zh.write(f"{zip_hash}  {output_zip.name}\n")

    print("Done.")
    print(f"Created: {output_zip}")
    print(f"Created: {manifest_path}")
    print(f"Created: {ziphash_path}")


Creating zip: /kaggle/working/saved_models.zip
  adding: best_detector_1024x1024_df.pth
  adding: best_detector_256x256_pgd.pth
  adding: best_detector_512x512_cw.pth
  adding: best_detector_1024x1024_bim.pth
  adding: best_detector_32x32_pgd.pth
  adding: best_detector_512x512_pgd.pth
  adding: best_detector_128x128_df.pth
  adding: best_detector_512x512_bim.pth
  adding: best_detector_128x128_fgsm.pth
  adding: best_detector_32x32_fgsm.pth
  adding: best_detector_1024x1024_pgd.pth
  adding: best_detector_64x64_pgd.pth
  adding: best_detector_64x64_df.pth
  adding: best_detector_64x64_bim.pth
  adding: best_detector_512x512_df.pth
  adding: best_detector_1024x1024_cw.pth
  adding: best_detector_256x256_bim.pth
  adding: best_detector_256x256_fgsm.pth
  adding: best_detector_512x512_fgsm.pth
  adding: best_detector_32x32_bim.pth
  adding: best_detector_64x64_cw.pth
  adding: best_detector_128x128_pgd.pth
  adding: best_detector_256x256_cw.pth
  adding: best_detector_128x128_bim.pth
  a